In [1]:
import numpy as np
import time
import warnings
import pandas as pd
import pickle

import quantpy as qp

from collections import defaultdict, Counter
from tqdm.notebook import tqdm

warnings.filterwarnings('ignore', category=np.ComplexWarning)

In [2]:
with open('../data/states.pkl', 'rb') as f:
    states = pickle.load(f)
states

,name,state,title,povm,n_meas
0,zero1,"Quantum object\narray([[1.+0.j, 0.+0.j],\n ...",$|0\rangle$,proj-set,1000
1,pure1,"Quantum object\narray([[0.85355+0.j , 0.25 ...",$\cos\frac{\pi}{8}|0\rangle + \sin\frac{\pi}{8...,proj-set,1000
2,fullymixed1,"Quantum object\narray([[0.5+0.j, 0. +0.j],\n ...",1-qubit fully mixed state,proj-set,1000
3,zero2,"Quantum object\narray([[1.+0.j, 0.+0.j, 0.+0.j...",$|00\rangle$,proj-set,10000
4,ghz2,"Quantum object\narray([[0.5+0.j, 0. +0.j, 0. +...",$|\Phi^+\rangle$,proj-set,10000
5,fullymixed2,"Quantum object\narray([[0.25+0.j, 0. +0.j, 0....",2-qubit fully mixed state,proj-set,10000
6,ghz3,"Quantum object\narray([[0.5+0.j, 0. +0.j, 0. +...",3-qubit GHZ state,proj-set,100000


In [3]:
with open('../data/processes.pkl', 'rb') as f:
    processes = pickle.load(f)
processes

,name,process,title,povm,n_meas
0,hadamard,Quantum channel with Choi matrix\narray([[ 0.5...,Hadamard gate,proj-set,1000
1,rxpi2,Quantum channel with Choi matrix\narray([[0.5+...,$RX\left(\frac{\pi}{2}\right)$,proj-set,1000
2,rypi2,Quantum channel with Choi matrix\narray([[ 0.5...,$RY\left(\frac{\pi}{2}\right)$,proj-set,1000
3,dep1,Quantum channel with Choi matrix\narray([[0.5+...,Fully depolarizing 1-qubit,proj-set,1000
4,cnot,Quantum channel with Choi matrix\narray([[1.+0...,CNOT,proj-set,100000
5,ms,Quantum channel with Choi matrix\narray([[ 0.5...,Mølmer–Sørensen gate,proj-set,100000


In [33]:
N_REPEATS = 10000
N_POINTS = 1000
CONFIDENCE_LEVELS = [0.5, 0.75, 0.9, 0.95, 0.99]

states_results = defaultdict(list)

for _, state in tqdm(states.iterrows()):
    tmg = qp.StateTomograph(state['state'])
    tmg.experiment(state['n_meas'], povm=state['povm'])
    interval = qp.WangInterval(tmg, method='bbox', n_points=N_POINTS)
    dist, CLs = interval()
    
    results = Counter()
    
    for _ in range(N_REPEATS):
        tmg.experiment(state['n_meas'], povm=state['povm'])
        state_hat = tmg.point_estimate()
        distance_hat = qp.hs_dst(state_hat, state['state'])
        for cl in CONFIDENCE_LEVELS:
            if distance_hat < dist[int(cl * N_POINTS)]:
                results[cl] += 1
    
    states_results['name'].append(state['name'])
    for key, value in results.items():
        states_results[key].append(value / N_REPEATS)

In [36]:
pd.DataFrame(states_results)

,name,0.5,0.75,0.9,0.95,0.99
0,zero1,0.8674,0.9343,0.9736,0.9879,0.9971
1,pure1,0.9629,0.9853,0.9961,0.9981,0.9999
2,fullymixed1,0.9453,0.9789,0.9939,0.9978,0.9998
3,zero2,1.0000,1.0000,1.0000,1.0000,1.0000
4,ghz2,1.0000,1.0000,1.0000,1.0000,1.0000
5,fullymixed2,1.0000,1.0000,1.0000,1.0000,1.0000
6,ghz3,1.0000,1.0000,1.0000,1.0000,1.0000


In [39]:
N_REPEATS = 1000
N_POINTS = 1000
CONFIDENCE_LEVELS = [0.5, 0.75, 0.9, 0.95, 0.99]

processes_results = defaultdict(list)

for _, process in tqdm(processes.iterrows()):
    tmg = qp.ProcessTomograph(process['process'])
    tmg.experiment(process['n_meas'], povm=process['povm'])
    interval = qp.HolderInterval(tmg, kind ='wang', wang_method='bbox', n_points=N_POINTS)
    dist, CLs = interval()
    
    results = Counter()
    
    for _ in range(N_REPEATS):
        tmg.experiment(process['n_meas'], povm=process['povm'])
        process_hat = tmg.point_estimate()
        distance_hat = qp.hs_dst(process_hat.choi, process['process'].choi)
        for cl in CONFIDENCE_LEVELS:
            if distance_hat < dist[int(cl * N_POINTS)]:
                results[cl] += 1
    
    processes_results['name'].append(process['name'])
    for key, value in results.items():
        processes_results[key].append(value / N_REPEATS)

In [40]:
pd.DataFrame(processes_results)

,name,0.5,0.75,0.9,0.95,0.99
0,hadamard,1.0,1.0,1.0,1.0,1.0
1,rxpi2,1.0,1.0,1.0,1.0,1.0
2,rypi2,1.0,1.0,1.0,1.0,1.0
3,dep1,1.0,1.0,1.0,1.0,1.0
4,cnot,1.0,1.0,1.0,1.0,1.0
5,ms,1.0,1.0,1.0,1.0,1.0
